<a href="https://colab.research.google.com/github/MuratCKoc/salesinsider/blob/main/ProphetsPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [232]:
# First setup the dependencies

#Our most important library is FB Prophet which will be using rule based learning
from fbprophet import Prophet
import pandas as pd
import numpy as np

In [233]:
# We then read the CSV file from GitHub
df = pd.read_csv('https://raw.githubusercontent.com/MuratCKoc/salesinsider/main/salesinsider/data/Groceries_dataset.csv')
# As the file has multiple columns, we select the columns that we will be using for the project.
data_df = df[['Date', 'itemDescription']]
data_df.head()

,Date,itemDescription
0,21-07-2015,tropical fruit
1,05-01-2015,whole milk
2,19-09-2015,pip fruit
3,12-12-2015,other vegetables
4,01-02-2015,whole milk


In [234]:
# We then use the get dummies function to convert each of the food items into an indicator.
# What this means is that all the data is converted into a single number (1).
cat_df = pd.get_dummies(data_df["itemDescription"])
# We then concatenate the data
final_df = pd.concat([data_df,cat_df],axis=1)
# After concatenation of the data we then group all the data by the dates which gives a table we can use to conduct our machine learning algorithm.
grouped_df = final_df.groupby("Date", as_index=False).sum()
grouped_df.head()

,Date,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,01-01-2014,2,0,0,0,0,0,0,0,0,...,0,0,2,2,0,0,0,2,4,0
1,01-01-2015,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,1,0,3,0,0
2,01-02-2014,0,0,0,0,0,0,0,0,2,...,0,0,2,1,0,1,0,1,3,0
3,01-02-2015,0,1,0,1,0,0,0,0,2,...,0,0,1,0,0,1,0,9,6,0
4,01-03-2014,0,0,0,0,0,0,0,0,0,...,0,0,1,3,0,1,0,1,1,0


In [235]:
# This is not needed.
columns = grouped_df.columns
print (len(columns))

168


In [236]:
# This is not needed.
master_df = pd.DataFrame()
predictions_records = dict()

In [237]:
for j in range(727, len(grouped_df.index)):
   # Predict thru all transactions.
  for i in range (1, len(grouped_df.columns)): 
    prophet_df = grouped_df[["Date", grouped_df.columns[i]]]
    prophet_df.columns = ["ds", 'y']
    prophet_df['ds'] = pd.to_datetime(prophet_df.ds,format='%d-%m-%Y') 
    model = Prophet()
    model.fit(prophet_df)
    future = model.make_future_dataframe(periods=213)
    forecast = model.predict(future)
    predicted_df = forecast[["ds", "yhat"]]
    predicted_df.yhat = predicted_df.yhat.round()

    predicted_ColName = grouped_df.columns[i]+"_Predicted"
    predicted_df = predicted_df.rename( columns={"ds": "Date", "yhat": predicted_ColName})

    print(predicted_df)
    #initialize container df
    if len(master_df.index) == 0:
      master_df = predicted_df.copy()
    else:
      #master_df = pd.merge(master_df, predicted_df, on="Date")
      colName = grouped_df.columns[i]
      master_df[predicted_ColName] = predicted_df[predicted_ColName]
    master_df[colName] = grouped_df[colName]

    if i == len(grouped_df.columns):
      break
    print (master_df)

  print(i)
print("ASD")

          1.0               0.0     0.0                    1.0   
2                   1.0               0.0     0.0                    1.0   
3                   0.0               0.0     0.0                    1.0   
4                   3.0              -0.0     0.0                    1.0   
..                  ...               ...     ...                    ...   
936                 NaN               0.0     NaN                    0.0   
937                 NaN               0.0     NaN                    0.0   
938                 NaN               0.0     NaN                    0.0   
939                 NaN               0.0     NaN                    0.0   
940                 NaN               0.0     NaN                    0.0   

     white bread  white wine_Predicted  white wine  
0            0.0                   0.0         0.0  
1            1.0                   0.0         0.0  
2            1.0                   0.0         0.0  
3            1.0                   0.

In [238]:
# This table shows us a consolidated view of the table with the output of machine learning applied here.
# Notice that we have double the number of columns.
master_df.head()

,Date,Instant food products_Predicted,zwieback,UHT-milk_Predicted,UHT-milk,abrasive cleaner_Predicted,abrasive cleaner,artif. sweetener_Predicted,artif. sweetener,baby cosmetics_Predicted,...,whisky,white bread_Predicted,white bread,white wine_Predicted,white wine,whole milk_Predicted,whole milk,yogurt_Predicted,yogurt,zwieback_Predicted
0,2014-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,3.0,2.0,2.0,4.0,0.0
1,2014-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,2.0,3.0,2.0,0.0,0.0
2,2014-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,3.0,0.0
3,2014-01-04,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,3.0,9.0,2.0,6.0,0.0
4,2014-01-05,0.0,0.0,1.0,0.0,0.0,0.0,-0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,3.0,1.0,2.0,1.0,0.0


In [239]:
# We then have to start cleaning up the table and remove all "NAN"s from the predicted columns. This will make it easier for visualization purposes.
clean_df = master_df
clean_df = clean_df.replace(np.nan,"")
clean_df

,Date,Instant food products_Predicted,zwieback,UHT-milk_Predicted,UHT-milk,abrasive cleaner_Predicted,abrasive cleaner,artif. sweetener_Predicted,artif. sweetener,baby cosmetics_Predicted,...,whisky,white bread_Predicted,white bread,white wine_Predicted,white wine,whole milk_Predicted,whole milk,yogurt_Predicted,yogurt,zwieback_Predicted
0,2014-01-01,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0,1.0,0,0.0,0,3.0,2,2.0,4,0.0
1,2014-01-02,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0,1.0,1,0.0,0,2.0,3,2.0,0,0.0
2,2014-01-03,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0,1.0,1,0.0,0,2.0,1,2.0,3,0.0
3,2014-01-04,0.0,0,0.0,1,0.0,0,0.0,1,0.0,...,0,1.0,1,0.0,0,3.0,9,2.0,6,0.0
4,2014-01-05,0.0,0,1.0,0,0.0,0,-0.0,0,0.0,...,0,1.0,1,0.0,0,3.0,1,2.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,2016-07-26,0.0,,0.0,,0.0,,-0.0,,0.0,...,,0.0,,0.0,,5.0,,2.0,,0.0
937,2016-07-27,0.0,,1.0,,0.0,,0.0,,0.0,...,,0.0,,0.0,,5.0,,2.0,,0.0
938,2016-07-28,0.0,,0.0,,0.0,,-0.0,,0.0,...,,0.0,,0.0,,5.0,,2.0,,0.0
939,2016-07-29,0.0,,0.0,,0.0,,0.0,,0.0,...,,0.0,,0.0,,5.0,,2.0,,0.0


In [253]:
# When tried to move the tables into SQL we received errors, the cause of this was the certain column names have parenthesis.
# The parentheses are escape strings which cause an error, so we have to replace them.
clean_df.rename(columns={'flower (seeds)' : 'flower seeds'}, inplace=True)
clean_df.rename(columns={'liquor (appetizer)':'liquor appetizer'}, inplace=True)
clean_df.rename(columns={'flower (seeds)_Predicted' : 'flower seeds_Predicted'}, inplace=True)
clean_df.rename(columns={'liquor (appetizer)_Predicted':'liquor appetizer_Predicted'}, inplace=True)

In [254]:
# This is to check if the strings have been changed.
list(clean_df)

['Date',
 'Instant food products_Predicted',
 'zwieback',
 'UHT-milk_Predicted',
 'UHT-milk',
 'abrasive cleaner_Predicted',
 'abrasive cleaner',
 'artif. sweetener_Predicted',
 'artif. sweetener',
 'baby cosmetics_Predicted',
 'baby cosmetics',
 'bags_Predicted',
 'bags',
 'baking powder_Predicted',
 'baking powder',
 'bathroom cleaner_Predicted',
 'bathroom cleaner',
 'beef_Predicted',
 'beef',
 'berries_Predicted',
 'berries',
 'beverages_Predicted',
 'beverages',
 'bottled beer_Predicted',
 'bottled beer',
 'bottled water_Predicted',
 'bottled water',
 'brandy_Predicted',
 'brandy',
 'brown bread_Predicted',
 'brown bread',
 'butter_Predicted',
 'butter',
 'butter milk_Predicted',
 'butter milk',
 'cake bar_Predicted',
 'cake bar',
 'candles_Predicted',
 'candles',
 'candy_Predicted',
 'candy',
 'canned beer_Predicted',
 'canned beer',
 'canned fish_Predicted',
 'canned fish',
 'canned fruit_Predicted',
 'canned fruit',
 'canned vegetables_Predicted',
 'canned vegetables',
 'cat fo

In [255]:
# We then proceed to move the table to SQL
# Import SQLAlchemy Dependencies 
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from config import database_url

In [256]:
# To update database with dataframe 
def Update_Database(df_name, table_name):
    engine = create_engine(database_url, echo=False)
    session = Session(engine)
    Base = automap_base()
    Base.prepare(engine, reflect=True)    
    cxn = engine.connect()
    df_name.to_sql(name=table_name, con=engine, if_exists='append', index=True)
    print(table_name + ' added')

In [257]:
Update_Database(clean_df, "predicted_table")

predicted_table added
